In [ ]:
#Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
#Importing tarining Dataset
dataset = pd.read_csv('tcd-ml-1920-group-income-train.csv',low_memory=False)
dataset = dataset.drop('Instance',axis=1)
#dataset = dataset.drop('Wears Glasses',axis=1)
#dataset = dataset.drop('Hair Color',axis=1)
#dataset = dataset.drop('Gender',axis=1)
#data_raw = dataset.drop(['Profession','Size of City'],axis=1)
dataset.info()

In [ ]:
#cleaning training dataset
dataset['Housing Situation']=dataset['Housing Situation'].replace('0','nA')
dataset['Satisfation with employer']=dataset['Satisfation with employer'].fillna('unknown')
dataset['Gender'] = dataset['Gender'].fillna('unknown')
dataset['Gender']=dataset['Gender'].replace('0','unknown')
dataset['Profession']=dataset['Profession'].fillna('unknown')
dataset['University Degree']=dataset['University Degree'].fillna('unknown')
dataset['University Degree']=dataset['University Degree'].replace('0','unknown')
dataset['Year of Record']=dataset['Year of Record'].fillna(dataset['Year of Record'].median())
dataset=dataset.drop('Hair Color',axis=1)
dataset['Work Experience in Current Job [years]']=dataset['Work Experience in Current Job [years]'].replace('#NUM!',-1)
dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'])
dataset['Work Experience in Current Job [years]']=dataset['Work Experience in Current Job [years]'].replace(-1,dataset['Work Experience in Current Job [years]'].median()

In [ ]:
#importing test dataset
test3 = pd.read_csv("tcd-ml-1920-group-income-test.csv",low_memory=False)

In [ ]:
#cleaning test data
test3['Housing Situation']=test3['Housing Situation'].replace('0','nA')
test3['Satisfation with employer']=test3['Satisfation with employer'].fillna('unknown')
test3['Gender'] = test3['Gender'].fillna('unknown')
test3['Gender']=test3['Gender'].replace('0','unknown')
test3['Profession']=test3['Profession'].fillna('unknown')
test3['University Degree']=test3['University Degree'].fillna('unknown')
test3['University Degree']=test3['University Degree'].replace('0','unknown')
test3['Year of Record']=test3['Year of Record'].fillna(dataset['Year of Record'].median())
test3=test3.drop('Hair Color',axis=1)
test3['Work Experience in Current Job [years]']=test3['Work Experience in Current Job [years]'].replace('#NUM!',-1)
test3['Work Experience in Current Job [years]']=pd.to_numeric(test3['Work Experience in Current Job [years]'])
test3['Work Experience in Current Job [years]']=test3['Work Experience in Current Job [years]'].replace(-1,dataset['Work Experience in Current Job [years]'].median())
test3['Country'] = test3['Country'].fillna('abc')

In [ ]:

#target encoding
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:

# Target encode
trn, sub = target_encode(trn_df["Housing Situation"], 
                         sub_df["Housing Situation"], 
                         target=trn_df['Total Yearly Income [EUR]'], 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)
trn_df['Housing Situation'] = trn
sub_df['Housing Situation'] = sub
trn, sub = target_encode(trn_df["Satisfation with employer"], 
                         sub_df["Satisfation with employer"], 
                         target=trn_df['Total Yearly Income [EUR]'], 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)
trn_df["Satisfation with employer"] = trn
sub_df["Satisfation with employer"] = sub
trn, sub = target_encode(trn_df["Gender"], 
                         sub_df["Gender"], 
                         target=trn_df['Total Yearly Income [EUR]'], 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)
trn_df["Gender"] = trn
sub_df["Gender"] = sub
trn, sub = target_encode(trn_df["Country"], 
                         sub_df["Country"], 
                         target=trn_df['Total Yearly Income [EUR]'], 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)
trn_df["Country"] = trn
sub_df["Country"] = sub
trn, sub = target_encode(trn_df["Profession"], 
                         sub_df["Profession"], 
                         target=trn_df['Total Yearly Income [EUR]'], 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)
trn_df["Profession"] = trn
sub_df["Profession"] = sub
trn, sub = target_encode(trn_df["University Degree"], 
                         sub_df["University Degree"], 
                         target=trn_df['Total Yearly Income [EUR]'], 
                         min_samples_leaf=100,
                         smoothing=10,
                         noise_level=0.01)
trn.head(10)
trn_df["University Degree"] = trn
sub_df["University Degree"] = sub


In [ ]:

X = trn_df.drop(['Total Yearly Income [EUR]'], axis=1)
y = trn_df.iloc[:,-1]

In [ ]:

XTrain_final = X
XTest_final = sub_df
YTrain_final = y


In [ ]:
#Seperating test and validation set
from sklearn.model_selection import train_test_split
xTrain,xValidation,yTrain,yValidation = train_test_split(XTrain_final,YTrain_final,test_size=0.2,random_state=1234) 

In [ ]:
#creating LightGBM input matrix
import lightgbm as lgb
d_train = lgb.Dataset(xTrain, label=yTrain)

In [ ]:
#defining parameter
params = {'max_depth': 20, 'learning_rate': 0.002, "boosting": "gbdt", "bagging_seed": 11, "metric": 'mae', "verbosity": -1}

In [ ]:
#Training Model
clf = lgb.train(params, d_train, 100000)

In [ ]:
#Predicting validation set
y_predicted = clf.predict(xValidation)

In [ ]:
#Checking Error
from sklearn.metrics import mean_absolute_error
mean_absolute_error(yValidation, y_predicted)

In [ ]:
test1 = pd.read_csv("tcd-ml-1920-group-income-test.csv",low_memory=False)

In [ ]:
#Predidicting Kaggle Test data
y_predicted = clf.predict(XTest_final)

In [ ]:
#Creating submission File
submission = pd.DataFrame()
submission['Instance'] = test1['Instance']
submission['Total Yearly Income [EUR]'] = y_predicted
submission.to_csv("LightGBM_model.csv")